In [149]:
import numpy as np
import pandas as pd
import random
import statistics

random.seed(42)
np.random.seed(42)

array = pd.read_csv('var3.csv').values
std = 7
variance = std ** 2
mu = 0.001 
n = len(array)
m_0 = 1  
s_0_2 = 1

In [150]:
def find_best_s2_m(array, variance, n, m_0, s_0_2, mu, max_iters=1000000): # Функция поиска оптимальных m и s2

    m = m_0
    s_2 = s_0_2
    list_of_m0 = []
    list_of_s2 = []

    for _ in range(max_iters):
        for_slice = random.randint(1, n - 50)
        x_ = array[for_slice:for_slice + 50].mean()

        m = ((n * x_ / variance) + (m / s_2)) / ((n / variance) + (1 / s_2))
        s_2 = 1 / ((n / variance) + (1 / s_2))

        list_of_m0.append(m)
        list_of_s2.append(s_2)

        if len(list_of_m0) > 5000:  # Проверяем сходимость по последним 5000 значениям 
            stddev_m = statistics.stdev(list_of_m0[-5000:])
            if stddev_m <= mu:
                break

    return statistics.mean(list_of_m0), statistics.mean(list_of_s2)


In [151]:
def estimate_mean(array, variance, n, optimal_m0, optimal_s_2, mu, max_iters=1000000): # Сама фунция вариационного выхода с оптимальными m и s2

    m = optimal_m0
    s_2 = optimal_s_2
    list_of_ms = []

    for i in range(max_iters):
        for_slice = random.randint(1, n - 50)
        x_ = array[for_slice:for_slice + 50].mean()

        m = ((n * x_ / variance) + (m / s_2)) / ((n / variance) + (1 / s_2))
        s_2 = 1 / ((n / variance) + (1 / s_2))

        list_of_ms.append(m)

        if len(list_of_ms) > 5000:  # Проверяем сходимость по последним 5000 значениям
            stddev_m = statistics.stdev(list_of_ms[-5000:])
            if stddev_m <= mu:
                return i + 1, statistics.mean(list_of_ms), stddev_m

    return max_iters, statistics.mean(list_of_ms), statistics.stdev(list_of_ms) # Если не достигнута сходимость за max_iters итераций

In [152]:
optimal_m0, optimal_s_2 = find_best_s2_m(array, variance, n, m_0, s_0_2, mu)

iterations, estimated_mean, estimated_stddev = estimate_mean(array, variance, n, optimal_m0, optimal_s_2, mu)

print('Количество эпох: ', iterations)
print('Полученное среднее значение нормального отклонения в генеграции данных: ', estimated_mean)
print('Ошибка между нормальными отклонениями: ', estimated_stddev)

Количество эпох:  18296
Полученное среднее значение нормального отклонения в генеграции данных:  17.737154816709275
Ошибка между нормальными отклонениями:  0.00099933685316085
